# Fluent Python


Reading fluent python by Luciano Ramalho this is a collection of my favourite bits.


### Python data model - bool and `__bool__`


The python data model allows you to implement special methods that make custom types behave like the builtin types.


In [1]:
class Example:
    def __len__(self):
        print(f"running __len__")
        return 10

    def __repr__(self):
        print(f"running __repr__")
        return f"Example()"

    def __str__(self):
        print(f"running __str__")
        return f"<Example Python object"

    def __bool__(self):
        print(f"running __bool__")
        return True


In [2]:
example = Example()
example


running __repr__


Example()

In [3]:
len(example)


running __len__


10

In [4]:
if example:
    pass


running __bool__


To check the whether `example` is truthy it calls effectively calls `bool(example)` which looks for `example.__bool__()`
If there is no `__bool__` python tries to invoke `__len__` and if that returns 0 then `False` else `True`.


### `__repr__` and `__str__`


`__repr__` - unambiguous for developers.  
`__str__` - for the end user.


### Unpacking sequences


In [5]:
person_data = ("Simon", "Ward-Jones", 1, 2, 3)
name, surname, *numbers = person_data
print(name, surname, numbers, sep="\n")


Simon
Ward-Jones
[1, 2, 3]


In [6]:
# Unpacking into a list
person_data = [*person_data]
person_data


['Simon', 'Ward-Jones', 1, 2, 3]

### Bad matrix


In [7]:
matrix = [[None] * 3] * 3
matrix[0][0] = 1
matrix


[[1, None, None], [1, None, None], [1, None, None]]

In [8]:
# you probably want:
matrix = []
for _ in range(3):
    matrix.append([None] * 3)
matrix[0][0] = 1
matrix


[[1, None, None], [None, None, None], [None, None, None]]

### Key in min, max and sorted


In [9]:
numbers = [3, "5", 1, "10", 2, "4", "6"]


In [10]:
print(sorted(numbers, key=int))
print(min(numbers, key=int))


[1, 2, 3, '4', '5', '6', '10']
1


In [11]:
### Unpacking mappings


In [12]:
person_data_1 = {"name": "Simon", "surname": "Ward-Jones", "age": 20}
person_data_2 = {
    "name": "Simon",
    "surname": "Ward-Jones",
    "age": 30,
    "height": 2,
    "weight": 3,
}


In [13]:
{**person_data_1, **person_data_2}


{'name': 'Simon', 'surname': 'Ward-Jones', 'age': 30, 'height': 2, 'weight': 3}

In [14]:
person_data_1 | person_data_2


{'name': 'Simon', 'surname': 'Ward-Jones', 'age': 30, 'height': 2, 'weight': 3}

In [15]:
### Dictionary comprehension


In [16]:
country_population_pairs = [
    ("China", 1409517397),
    ("India", 1324171354),
    ("United States", 325234000),
    ("Indonesia", 255461563),
    ("Brazil", 210147125),
    ("Pakistan", 197015955),
    ("Nigeria", 206139589),
    ("Bangladesh", 164895000),
    ("Russia", 146585181),
    ("Japan", 126470000),
]


In [17]:
# create a dictionary of country populations
country_populations = {
    country: population for country, population in country_population_pairs
}


### Dictionary setdefault method


In [18]:
person_data_1 = {"name": "Simon", "surname": "Ward-Jones", "age": 20}
person_data_2 = {
    "name": "Simon",
    "surname": "Ward-Jones",
    "age": 30,
    "height": 2,
    "weight": 3,
}


In [19]:
# set default looks up the key and returns the value if it exists
# if it doesn't exist, it sets the key to the default value and returns the default value
person_data_1.setdefault("name", "John")


'Simon'

In [20]:
# This is really useful for something like this:
person_data_1.setdefault("animals", {}).setdefault("cats", []).append("cat")
person_data_1


{'name': 'Simon',
 'surname': 'Ward-Jones',
 'age': 20,
 'animals': {'cats': ['cat']}}

### Dictionary views and mappings work like sets


In [21]:
person_data_1 = {"name": "Simon", "surname": "Ward-Jones", "age": 20}
person_data_2 = {
    "name": "Simon",
    "surname": "Ward-Jones",
    "age": 30,
    "height": 2,
    "weight": 3,
}
# finding common keys
person_data_1.keys() & person_data_2.keys()


{'age', 'name', 'surname'}

### Unicode names


In [22]:
from unicodedata import name
import random


In [23]:
n = 0
while n < 5:
    unicode_point = chr(random.randint(0, 0x10FFFF))  # pick a random unicode point
    if unicode_name := name(unicode_point, ""):
        print(unicode_point, unicode_name)  # print the point and the name
        n += 1


𘮴 KHITAN SMALL SCRIPT CHARACTER-18BB4
潚 CJK UNIFIED IDEOGRAPH-6F5A
𝤍 SIGNWRITING STRIKE BETWEEN
ﶢ ARABIC LIGATURE TEH WITH KHAH WITH ALEF MAKSURA FINAL FORM
ם HEBREW LETTER FINAL MEM


### string translation


In [24]:
translation = str.maketrans("simon", "abcde")


In [25]:
"hello simon".translate(translation)


'helld abcde'

### Dataclasses save you a lot of typing


In [26]:
from dataclasses import dataclass


In [27]:
from dataclasses import field
from typing import Optional


@dataclass
class Book:
    title: str
    author: Optional[str] = "unknown"
    pages: int = 0
    price: float = field(default=0.0, repr=False)

    def __post_init__(self):
        self.long_title = self.title + " by " + self.author


In [28]:
book = Book(title="Python Programming", author="Simon Ward-Jones", pages=300)
book


Book(title='Python Programming', author='Simon Ward-Jones', pages=300)

### Variables are not boxes 📦 🏷


In [29]:
variable = ["📦 ", "🏷"]
other_variable = variable
other_variable.append("📦")
variable  # say what...


['📦 ', '🏷', '📦']

# Fin.